Question 3: We would like to know something about our populations of users, in particular, we would like to have a good model of whether or not a player will continue contributing given past participation. 

In [2]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate



alt.data_transformers.enable('vegafusion')

set_config(transform_output="pandas")

In [3]:
players_messy=pd.read_csv('players.csv')
sessions_messy=pd.read_csv('sessions.csv')

In [4]:
sessions_messy[['start_date','start_time']]=sessions_messy['start_time'].str.split(' ', expand=True)
sessions_messy[['end_date','end_time']]=sessions_messy['end_time'].str.split(' ', expand=True)

In [5]:
sessions=sessions_messy.drop(columns=['original_start_time','original_end_time', 'start_date', 'end_date'])
sessions

,hashedEmail,start_time,end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,18:12,18:24
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,23:33,23:46
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,17:34,17:57
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,03:22,03:58
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,16:01,16:12
...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,23:01,23:07
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,04:08,04:19
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,15:36,15:57
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,06:15,06:22


In [6]:
sessions['start_time']=pd.to_datetime(sessions['start_time'], format='%H:%M')
sessions['end_time']=pd.to_datetime(sessions['end_time'], format='%H:%M')

#created new columns for start and end time but the data type is numerical
sessions['start_time_numerical']=sessions['start_time'].dt.hour+sessions['start_time'].dt.minute/60
sessions['end_time_numerical']=sessions['end_time'].dt.hour+sessions['end_time'].dt.minute/60

#rounded to 3 decimals
sessions['start_time_numerical']=sessions['start_time_numerical'].round(3)
sessions['end_time_numerical']=sessions['end_time_numerical'].round(3)

#made a new column for the elapsed session time in hours
sessions['session_length']=(sessions['end_time_numerical']-sessions['start_time_numerical'])
sessions

,hashedEmail,start_time,end_time,start_time_numerical,end_time_numerical,session_length
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,1900-01-01 18:12:00,1900-01-01 18:24:00,18.200,18.400,0.200
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1900-01-01 23:33:00,1900-01-01 23:46:00,23.550,23.767,0.217
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,1900-01-01 17:34:00,1900-01-01 17:57:00,17.567,17.950,0.383
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,1900-01-01 03:22:00,1900-01-01 03:58:00,3.367,3.967,0.600
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1900-01-01 16:01:00,1900-01-01 16:12:00,16.017,16.200,0.183
...,...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1900-01-01 23:01:00,1900-01-01 23:07:00,23.017,23.117,0.100
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,1900-01-01 04:08:00,1900-01-01 04:19:00,4.133,4.317,0.184
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,1900-01-01 15:36:00,1900-01-01 15:57:00,15.600,15.950,0.350
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,1900-01-01 06:15:00,1900-01-01 06:22:00,6.250,6.367,0.117


In [7]:
sessions['start_time_12h'] = sessions['start_time'].dt.strftime('%I:%M %p')
sessions['end_time_12h'] = sessions['end_time'].dt.strftime('%I:%M %p')

sessions['start_12h_numerical']=sessions['start_time_12h'].dt.hour+sessions['start_time_12h'].dt.minute/60
sessions['end_12h_numerical']=sessions['end_time_12h'].dt.hour+sessions['end_time_12h'].dt.minute/60

#made a new column for the elapsed session time in hours
sessions['session_length']=(sessions['end_12h_numerical']-sessions['start_12h_numerical'])
sessions

AttributeError: Can only use .dt accessor with datetimelike values

In [8]:
sessions=sessions.drop(columns=['start_time_numerical','end_time_numerical','start_time','end_time'])
sessions

,hashedEmail,session_length,start_time_12h,end_time_12h
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,0.200,06:12 PM,06:24 PM
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,0.217,11:33 PM,11:46 PM
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,0.383,05:34 PM,05:57 PM
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,0.600,03:22 AM,03:58 AM
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,0.183,04:01 PM,04:12 PM
...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,0.100,11:01 PM,11:07 PM
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,0.184,04:08 AM,04:19 AM
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,0.350,03:36 PM,03:57 PM
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,0.117,06:15 AM,06:22 AM


In [9]:
sessions_negative = sessions[sessions["session_length"]<0]
sessions_negative

,hashedEmail,session_length,start_time_12h,end_time_12h
44,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,-23.150,11:23 PM,12:14 AM
51,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,-21.433,09:26 PM,12:00 AM
52,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,-23.117,11:30 PM,12:23 AM
61,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,-22.116,10:38 PM,12:31 AM
80,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,-21.534,11:22 PM,01:50 AM
...,...,...,...,...
1490,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,-22.350,11:26 PM,01:05 AM
1497,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,-23.033,11:44 PM,12:42 AM
1509,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,-21.700,10:33 PM,12:51 AM
1511,b622593d2ef8b337dc554acb307d04a88114f2bf453b18...,-19.750,09:54 PM,02:09 AM


In [35]:
sessions_group=sessions.groupby('hashedEmail').mean(numeric_only=True).reset_index()
sessions_group.round(2)

,hashedEmail,session_length
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,-11.12
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0.50
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.18
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,0.54
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,0.58
...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.27
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1.33
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,-0.05
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0.15


In [36]:
sessions_group_all=sessions.groupby('hashedEmail',as_index=False).agg({
    'session_length': 'mean',
    'start_time_12h':'first',
    'end_time_12h':'first'
}
)
sessions_group_all.round(2)

,hashedEmail,session_length,start_time_12h,end_time_12h
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,-11.12,11:12 PM,12:13 AM
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0.50,04:28 AM,04:58 AM
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.18,09:01 PM,09:12 PM
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,0.54,09:59 PM,10:36 PM
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,0.58,06:27 PM,07:14 PM
...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.27,11:26 PM,11:42 PM
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1.33,10:26 PM,11:46 PM
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,-0.05,07:12 AM,07:21 AM
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0.15,03:33 PM,03:42 PM


In [17]:
players_messy

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [18]:
players=players_messy.drop(columns=['individualId','organizationName'])
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21
...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17


In [19]:
players_sessions=sessions_group_all.merge(players, on='hashedEmail')
players_sessions

,hashedEmail,session_length,start_time,end_time,experience,subscribe,played_hours,name,gender,age
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,-11.116500,23:12,00:13,Regular,True,1.5,Isaac,Male,20
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0.500000,04:28,04:58,Pro,False,0.4,Lyra,Male,21
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.183000,21:01,21:12,Beginner,True,0.1,Osiris,Male,17
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,0.536077,21:59,22:36,Regular,True,5.6,Winslow,Male,17
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,0.583500,18:27,19:14,Pro,True,1.0,Knox,Male,17
...,...,...,...,...,...,...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.267000,23:26,23:42,Amateur,True,0.2,Gemna,Male,27
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1.334000,22:26,23:46,Pro,True,1.2,Sakura,Male,17
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,-0.051881,07:12,07:21,Amateur,True,56.1,Dana,Male,23
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0.150000,15:33,15:42,Amateur,True,0.1,Fatima,Male,17


In [20]:
new_column_order = ['name','start_time','end_time','session_length','played_hours','subscribe','experience']


players_sessions= players_sessions[new_column_order]
players_sessions

,name,start_time,end_time,session_length,played_hours,subscribe,experience
0,Isaac,23:12,00:13,-11.116500,1.5,True,Regular
1,Lyra,04:28,04:58,0.500000,0.4,False,Pro
2,Osiris,21:01,21:12,0.183000,0.1,True,Beginner
3,Winslow,21:59,22:36,0.536077,5.6,True,Regular
4,Knox,18:27,19:14,0.583500,1.0,True,Pro
...,...,...,...,...,...,...,...
120,Gemna,23:26,23:42,0.267000,0.2,True,Amateur
121,Sakura,22:26,23:46,1.334000,1.2,True,Pro
122,Dana,07:12,07:21,-0.051881,56.1,True,Amateur
123,Fatima,15:33,15:42,0.150000,0.1,True,Amateur


In [45]:
ps=sessions_group.merge(players, on='hashedEmail')
ps

,hashedEmail,session_length,start_time_numerical,end_time_numerical,experience,subscribe,played_hours,name,gender,age
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,-11.116500,11.783500,0.667000,Regular,True,1.5,Isaac,Male,20
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,0.500000,4.467000,4.967000,Pro,False,0.4,Lyra,Male,21
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,0.183000,21.017000,21.200000,Beginner,True,0.1,Osiris,Male,17
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,0.535923,10.574231,11.110308,Regular,True,5.6,Winslow,Male,17
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,0.583000,20.066500,20.650000,Pro,True,1.0,Knox,Male,17
...,...,...,...,...,...,...,...,...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,0.267000,23.433000,23.700000,Amateur,True,0.2,Gemna,Male,27
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1.333000,22.433000,23.767000,Pro,True,1.2,Sakura,Male,17
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,-0.051881,10.881135,10.829255,Amateur,True,56.1,Dana,Male,23
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,0.150000,15.550000,15.700000,Amateur,True,0.1,Fatima,Male,17


NameError: name 'index' is not defined

In [52]:
#need to make the 12H columns numerical to make the new column for session length
#sessions['start_time_numerical']=sessions['start_time'].dt.hour+sessions['start_time'].dt.minute/60
#sessions['end_time_numerical']=sessions['end_time'].dt.hour+sessions['end_time'].dt.minute/60

#got rid of 1900-01-01 numbers before the time 
#sessions['start_time']=sessions['start_time'].dt.strftime('%H:%M')
#sessions['end_time']=sessions['end_time'].dt.strftime('%H:%M')